In [14]:
import numpy as np
from sympy import * #var, Lambda, exp, log, sin, cos, tan, sqrt, ln

In [15]:
y12 = -1/(0.01+0.05j)
y21 = -1/(0.01+0.05j)
y11 = -y12
y22 = -y12

#Matriz de admitância
Y = np.empty((2), dtype=complex) #ordem da matriz
Y = np.array([[y11, y12], [y21, y22]])
G = Y.real
B = Y.imag

#Variáveis de entrada
teta2, V2 = var('teta2 V2')
x = np.array([[teta2],[V2]])
teta1 = 0
V1 = 1

# Potências esperadas nas barras 
P_esp_2 = -1
Q_esp_2 = -0

# Vetor de potencias
P2 = Lambda((teta2, V2), V2*V1*(G[1,0]*cos(teta2-teta1) + B[1,0]*sin(teta2-teta1)) + G[1,1]*V2**2)
Q2 = Lambda((teta2, V2), V2*V1*(G[1,0]*sin(teta2-teta1) - B[1,0]*cos(teta2-teta1)) + B[1,1]*V2**2)
y = np.array([[P_esp_2 - P2(teta2, V2)], [Q_esp_2 - Q2(teta2, V2)]])

# Matriz Jacobiana
H22 = Lambda((teta2, V2), diff(P2(teta2, V2),teta2)).simplify()
N22 = Lambda((teta2, V2), diff(P2(teta2, V2),V2)).simplify()
M22 = Lambda((teta2, V2), diff(Q2(teta2, V2),teta2)).simplify()
L22 = Lambda((teta2, V2), diff(Q2(teta2, V2),V2)).simplify()
J = np.array([[H22(teta2, V2), N22(teta2, V2)], [M22(teta2, V2), L22(teta2, V2)]])

# Vetor inicial
x0 = np.array([[0], [1]])

# Erro
e = 1e-6

# Numeros de iterações
iter = 0

In [16]:
# Solução

print('\nY = ', Y)
print('\nG = ', G)
print('\nB = ', B)
print('\n')

for i in range(len(y)):
    print('y[',i,'] = ', y[i])
print('\n')

for i in range(len(x)):
    print('x[',i,'] = ', x[i])
print('\n')

for i in range(len(J)):
    for j in range(len(J)):
        print('J[',i,'][',j,'] = ', J[i][j])
print('\n')


Y =  [[ 3.84615385-19.23076923j -3.84615385+19.23076923j]
 [-3.84615385+19.23076923j  3.84615385-19.23076923j]]

G =  [[ 3.84615385 -3.84615385]
 [-3.84615385  3.84615385]]

B =  [[-19.23076923  19.23076923]
 [ 19.23076923 -19.23076923]]


y[ 0 ] =  [-3.84615384615385*V2**2 - V2*(19.2307692307692*sin(teta2) - 3.84615384615385*cos(teta2)) - 1]
y[ 1 ] =  [19.2307692307692*V2**2 - V2*(-3.84615384615385*sin(teta2) - 19.2307692307692*cos(teta2))]


x[ 0 ] =  [teta2]
x[ 1 ] =  [V2]


J[ 0 ][ 0 ] =  V2*(3.84615384615385*sin(teta2) + 19.2307692307692*cos(teta2))
J[ 0 ][ 1 ] =  7.69230769230769*V2 + 19.2307692307692*sin(teta2) - 3.84615384615385*cos(teta2)
J[ 1 ][ 0 ] =  V2*(19.2307692307692*sin(teta2) - 3.84615384615385*cos(teta2))
J[ 1 ][ 1 ] =  -38.4615384615385*V2 - 3.84615384615385*sin(teta2) - 19.2307692307692*cos(teta2)




In [45]:
def passo2(x0, P2, Q2):
    delta_P2 = P_esp_2 - P2(teta2, V2)
    delta_Q2 = Q_esp_2 - Q2(teta2, V2)
    teta22 = x0[0]
    V22 = x0[1]
    delta_P2 = P_esp_2 - P2(teta22, V22)
    delta_Q2 = Q_esp_2 - Q2(teta22, V22)
    x1 = (P2(teta2, V2), (teta2, V2), (x0[0], x0[1]))
    x2 = (Q2(teta2, V2), (teta2, V2), (x0[0], x0[1]))

    xn = np.array([x1, x2])
    deltay = np.array([delta_P2, delta_Q2])

    print('\nxn = ', xn.T)
    print('\ndelta = ', deltay.T)

    passo3(xn, deltay)

def passo3(xn, deltay):
    if (max(deltay) <= e):
        passo7(xn)
    else:
        passo4_5_6(xn, deltay, J)

def passo4_5_6(xn, deltay, J):
    Jn = np.array([[J[0][0](xn[0], xn[1]), J[0][1](xn[0], xn[1])], [J[1][0](xn[0], xn[1]), J[1][1](xn[0], xn[1])]])
    print('\nJn = ', Jn)

    Jn_inv = np.linalg.inv(Jn)
    print('\nJn_inv = ', Jn_inv)
    deltax = Jn_inv*deltay
    print('\ndelta_x = ', deltax.T)

    x0[0] = x[0] + deltax[0]
    x0[1] = x[1] + deltax[1]
     
    iter += 1
    print('\nIteração ', iter)
    passo2(x0, P2, Q2)
    
'''def passo7(xn):
    print('\nSolução:')
    print('\nxn = ', xn.T)
    print('\nx = ', x0.T)
    print('\n')'''

"def passo7(xn):\n    print('\nSolução:')\n    print('\nxn = ', xn.T)\n    print('\nx = ', x0.T)\n    print('\n')"

In [47]:
teta2, V2 = var('teta2 V2')
teta1 = 0
V1 = 1

P2 = Lambda((teta2, V2), V2*V1*(G[1,0]*cos(teta2-teta1) + B[1,0]*sin(teta2-teta1)) + G[1,1]*V2**2).simplify()
Q2 = Lambda((teta2, V2), V2*V1*(G[1,0]*sin(teta2-teta1) - B[1,0]*cos(teta2-teta1)) + B[1,1]*V2**2).simplify()

print('P2 = ', P2(teta2, V2))
print('Q2 = ', Q2(teta2, V2))

P_esp_2 = -1
Q_esp_2 = -0

teta2_inicial = 0
V2_inicial = 1

H = Lambda((teta2, V2), diff(P2(teta2, V2),teta2)).simplify()
N = Lambda((teta2, V2), diff(P2(teta2, V2),V2)).simplify()
M = Lambda((teta2, V2), diff(Q2(teta2, V2),teta2)).simplify()
L = Lambda((teta2, V2), diff(Q2(teta2, V2),V2)).simplify()

print('H = ', H(teta2, V2))
print('N = ', N(teta2, V2))
print('M = ', M(teta2, V2))
print('L = ', L(teta2, V2))

delta_P2 = P_esp_2 - P2(teta2, V2)
delta_Q2 = Q_esp_2 - Q2(teta2, V2)

print('delta_P2 = ', M(teta2, V2))
print('delta_Q2 = ', Q2)


P2 =  V2*(3.84615384615385*V2 + 19.2307692307692*sin(teta2) - 3.84615384615385*cos(teta2))
Q2 =  -V2*(19.2307692307692*V2 + 3.84615384615385*sin(teta2) + 19.2307692307692*cos(teta2))
H =  V2*(3.84615384615385*sin(teta2) + 19.2307692307692*cos(teta2))
N =  7.69230769230769*V2 + 19.2307692307692*sin(teta2) - 3.84615384615385*cos(teta2)
M =  V2*(19.2307692307692*sin(teta2) - 3.84615384615385*cos(teta2))
L =  -38.4615384615385*V2 - 3.84615384615385*sin(teta2) - 19.2307692307692*cos(teta2)
delta_P2 =  V2*(19.2307692307692*sin(teta2) - 3.84615384615385*cos(teta2))
delta_Q2 =  Lambda((teta2, V2), -V2*(19.2307692307692*V2 + 3.84615384615385*sin(teta2) + 19.2307692307692*cos(teta2)))


In [ ]:
l = [[P2(teta2, V2)], [Q2(teta2, V2)]]
for i in range(len(l)):
    print(l[i],len(l) )

[V2*(3.84615384615385*V2 + 19.2307692307692*sin(teta2) - 3.84615384615385*cos(teta2))] 2
[-V2*(19.2307692307692*V2 + 3.84615384615385*sin(teta2) + 19.2307692307692*cos(teta2))] 2
